# Extract the most populated pixel in each administrative unit

This notebook tries to do better than centroids for admin units where you don't have spatial data for the capital, but want to measure from such. It finds the most populated pixel in that admin unit and treats that as the capital instead. Note this won't work if the capital is not the most populated place for any reason.

In [2]:
import os, sys
import pandas as pd
import geopandas as gpd
import numpy as np

import pprint

import rasterio
from rasterio import features

/Users/robert/.pyenv/versions/geo/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
from geopandas import GeoSeries
from shapely.geometry import Point

#### Setup parameters

In [4]:
geo_pth = r"/Users/robert/Documents/Jobs/WB/Pakistan/Data"
pop_pth = r"/Users/robert/Documents/Jobs/WB/Pakistan/Data/Population"
data_pth = r'../../data/vect_out'

In [5]:
# geo_pth = r"P:\BGD\GEO"
# pop_pth = r'P:\BGD\GEO\Population\WorldPop'

#### Import the points, spatial join admin data

In [6]:
adm = gpd.read_file(os.path.join(geo_pth,"Boundaries/KP_Analysis_Focus_Districts.gpkg"),driver="GPKG")

/Users/robert/.pyenv/versions/geo/lib/python3.9/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [26]:
adm = adm.to_crs(4326)

In [8]:
adm.rename({'PROVINCE':'Province','DISTRICT':'District'},axis=1,inplace=True)

In [10]:
pop = gpd.read_file(os.path.join(pop_pth,"wp_KPAnalysis_2020_100m_unconstrained_pts.gpkg"),driver="GPKG")

/Users/robert/.pyenv/versions/geo/lib/python3.9/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [28]:
pop = gpd.sjoin(pop[['VALUE','geometry']],adm[['geometry','Province','District']],how='left',op='within')

In [29]:
pop.head(2)

,VALUE,geometry,index_right,Province,District
0,0.039197,POINT (72.57250 36.83333),12,KHYBER PAKHTUNKHWA,CHITRAL
1,0.038564,POINT (72.57333 36.83333),12,KHYBER PAKHTUNKHWA,CHITRAL


#### Filter down

In [30]:
# idx_adm2 = pop.groupby(['Division'])['VALUE'].transform(max) == pop['VALUE']
# pop_adm2_max = pop[idx_adm2]

In [31]:
# pop_adm2_max

In [32]:
idx_adm3 = pop.groupby(['District'])['VALUE'].transform(max) == pop['VALUE']
pop_adm3_max = pop[idx_adm3]

In [33]:
pop_adm3_max

,VALUE,geometry,index_right,Province,District
1365135,15.714732,POINT (71.77250 35.71667),12,KHYBER PAKHTUNKHWA,CHITRAL
2542272,3.907878,POINT (73.53667 35.53167),16,KHYBER PAKHTUNKHWA,UPPER KOHISTAN
3055652,50.624680,POINT (70.10000 33.89750),5,FATA,KURRAM AGENCY
3357044,4.993747,POINT (70.83667 33.66167),8,FATA,ORAKZAI AGENCY
3721658,25.882614,POINT (72.02000 34.94167),22,KHYBER PAKHTUNKHWA,UPPER DIR
3774884,75.144615,POINT (71.98083 34.89167),18,KHYBER PAKHTUNKHWA,LOWER DIR
3970865,46.952003,POINT (71.52500 34.73667),0,FATA,BAJAUR AGENCY
4448472,39.159988,POINT (71.43417 34.18417),6,FATA,MOHMAND AGENCY
4607409,200.074081,POINT (71.43833 33.90333),23,FATA,KHYBER AGENCY
5340769,43.811085,POINT (72.88167 34.92917),24,KHYBER PAKHTUNKHWA,LOWER KOHISTAN


In [34]:
# idx_adm4 = pop.groupby(["adm4_pcode","adm4_en"])['VALUE'].transform(max) == pop['VALUE']
# pop_adm4_max = pop[idx_adm4]

In [35]:
# pop_adm4_max

#### Export the points

In [36]:
# pop_adm2_max = pop_adm2_max.to_crs(4326)
pop_adm3_max = pop_adm3_max.to_crs(4326)
# pop_adm4_max = pop_adm4_max.to_crs(4326)

In [37]:
# pop_adm2_max.to_file(os.path.join(pop_pth,"adm2_maxpop_pt.gpkg"),driver="GPKG")
pop_adm3_max.to_file(os.path.join(pop_pth,"adm3_maxpop_pt.gpkg"),driver="GPKG")
# pop_adm4_max.to_file(os.path.join(pop_pth,"adm4_maxpop_pt.gpkg"),driver="GPKG")

In [38]:
# pop_adm2_max.to_csv(os.path.join(pop_pth,"adm2_maxpop_pt.csv"))
pop_adm3_max.to_csv(os.path.join(pop_pth,"adm3_maxpop_pt.csv"))
# pop_adm4_max.to_csv(os.path.join(pop_pth,"adm4_maxpop_pt.csv"))